# Install

In [1]:
!pip install  datasets

# Import

In [ ]:
#база
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import json

#НС
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split


#Обработка текстов

import razdel
from razdel import sentenize
from pymystem3 import Mystem
import spacy
import re
from wordcloud import WordCloud
from collections import Counter

#Hugging Face
from transformers import get_scheduler, AutoTokenizer, BertForTokenClassification, AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import T5ForConditionalGeneration, AutoModelForCausalLM
from datasets import Dataset, load_dataset

# Gazeta

In [ ]:
test = load_dataset("IlyaGusev/gazeta", split='test')
test = test.shuffle(seed=42).select(range(500))
print(test)

Dataset({
    features: ['text', 'summary', 'title', 'date', 'url'],
    num_rows: 500
})


# Экстрактивное рефер.

In [ ]:
model_name = "IlyaGusev/rubert_ext_sum_gazeta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForTokenClassification.from_pretrained(model_name).to("cuda")  # Отправляем на GPU
sep_token_id = tokenizer.sep_token_id

tokenizer_config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/889 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/709M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

In [ ]:
def generate_summaries(batch, tokenizer, model):
    sep_token = tokenizer.sep_token
    """Генерирует рефераты для батча статей"""
    batch_sentences = [list(razdel.sentenize(text)) for text in batch["text"]]
    batch_texts = [sep_token.join([s.text for s in sentences]) for sentences in batch_sentences]

    # Токенизация всего батча сразу
    inputs = tokenizer(
        batch_texts,
        max_length=500,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    ).to("cuda")  # Отправляем на GPU

    sep_masks = inputs["input_ids"] == tokenizer.sep_token_id

    # Устанавливаем чередующиеся token_type_ids
    current_token_type_ids = torch.zeros_like(inputs["input_ids"])
    for i in range(inputs["input_ids"].shape[0]):  # Для каждого примера в батче
        current_token_type_id = 0
        for pos, input_id in enumerate(inputs["input_ids"][i]):
            current_token_type_ids[i][pos] = current_token_type_id
            if input_id == tokenizer.sep_token_id:
                current_token_type_id = 1 - current_token_type_id
    inputs["token_type_ids"] = current_token_type_ids

    # Инференс модели
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[:, :, 1]  # Оставляем только логиты 1-го класса

    summaries = []
    for i, sentences in enumerate(batch_sentences):
        sep_positions = torch.where(sep_masks[i])[0]  # Индексы [SEP] токенов
        sentence_logits = logits[i, sep_positions]  # Берем логиты только на этих позициях
        top_indices = torch.argsort(sentence_logits, descending=True)[:3]  # Топ-3 предложения
        top_indices = sorted(top_indices.cpu().tolist())  # Сортируем в порядке появления
        summary = " ".join([sentences[idx].text for idx in top_indices])
        summaries.append(summary)

    return {"generated_summary": summaries}

>Эта модель - просто оценивает насколько важны предложения. Этот код выбирает ровно 3 самых вероятных предложения (может стоит увеличить). Кстати модель ни разу мне не выдала вероятность взять предложение (2 класс) > 50%

In [ ]:
res = test.map(lambda batch: generate_summaries(batch, tokenizer, model), batched=True, batch_size=64)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
df = res.to_pandas().loc[:, ['text', 'summary']]
df['rubert_ext_sum'] = res['generated_summary']
df.head(2)

,text,summary,rubert_ext_sum
0,Американские вооруженные силы должны быть гото...,Экс-президент США продолжает критиковать админ...,Американские вооруженные силы должны быть гото...
1,"Международный коллектив ученых из Гарварда , П...",Международный коллектив под руководством гарва...,"Международный коллектив ученых из Гарварда , П..."


In [ ]:
path = '/content/drive/MyDrive/Курсовые/Transformers 2/'
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)

# Абстрактивное рефер.

## T5

In [ ]:
def gen_batch(inputs, batch_size):
    batch_start = 0
    while batch_start < len(inputs):
        yield inputs[batch_start: batch_start + batch_size]
        batch_start += batch_size


def predict(
    model_name,
    input_records,
    output_file,
    max_source_tokens_count=600,
    batch_size=8
):
    device = "cuda" if torch.cuda.is_available() else "cpu"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

    predictions = []
    for batch in tqdm(gen_batch(input_records, batch_size)):
        texts = [r["text"] for r in batch]
        input_ids = tokenizer(
            texts,
            add_special_tokens=True,
            max_length=max_source_tokens_count,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )["input_ids"].to(device)

        output_ids = model.generate(
            input_ids=input_ids,
            no_repeat_ngram_size=4
        )
        summaries = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        predictions.extend(summaries)

    return predictions


pred = predict("IlyaGusev/rut5_base_sum_gazeta", list(test), "t5_predictions.txt")


0it [00:00, ?it/s]

In [ ]:
path = '/content/drive/MyDrive/Курсовые/Transformers 2/'
df = pd.read_excel(path + "Рефераты_моделей.xlsx")
df['rut5_abs_sum'] = pred
df.head(2)

,text,summary,rubert_ext_sum,rut5_abs_sum
0,Американские вооруженные силы должны быть гото...,Экс-президент США продолжает критиковать админ...,Американские вооруженные силы должны быть гото...,"Бывший глава Белого дома Дональд Трамп заявил,..."
1,"Международный коллектив ученых из Гарварда , П...",Международный коллектив под руководством гарва...,"Международный коллектив ученых из Гарварда , П...","Международный коллектив ученых из Гарварда, Пр..."


In [ ]:
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)

## RU_GPT3

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "IlyaGusev/rugpt3medium_sum_gazeta"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [ ]:
batch_size = 4
max_input_len = 600
max_new_tokens = 100
predictions = []

In [ ]:
model.eval()
with torch.no_grad():
    for batch in tqdm([test['text'][i:i+batch_size] for i in range(0, len(test), batch_size)]):
        texts = batch

        tokenized = tokenizer(
            texts,
            max_length=max_input_len,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )
        input_ids = tokenized["input_ids"].to(device)

        attention_mask = tokenized["attention_mask"].to(device)

        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,  # ← ВАЖНО
            max_new_tokens=max_new_tokens,
            no_repeat_ngram_size=4,
            pad_token_id=tokenizer.eos_token_id
        )

        for out in output:
            summary = tokenizer.decode(out, skip_special_tokens=False)
            try:
                summary = summary.split(tokenizer.sep_token)[1].split(tokenizer.eos_token)[0]
            except IndexError:
                summary = ""
            predictions.append(summary.strip())

  0%|          | 0/125 [00:00<?, ?it/s]

In [ ]:
path = '/content/drive/MyDrive/Курсовые/Transformers 2/'
path = '/content/drive/MyDrive/'
df = pd.read_excel(path + "Рефераты_моделей.xlsx")
df['gpt_abs_sum'] = predictions
df.head(2)

,text,summary,rubert_ext_sum,rut5_abs_sum,gpt_abs_sum
0,Американские вооруженные силы должны быть гото...,Экс-президент США продолжает критиковать админ...,Американские вооруженные силы должны быть гото...,"Бывший глава Белого дома Дональд Трамп заявил,...","Бывший президент США Дональд Трамп заявил, что..."
1,"Международный коллектив ученых из Гарварда , П...",Международный коллектив под руководством гарва...,"Международный коллектив ученых из Гарварда , П...","Международный коллектив ученых из Гарварда, Пр...",Ученые из Гарварда и Принстона объявили о нача...


In [ ]:
df.to_excel(path + "Рефераты_моделей.xlsx", index=False)